This notebook will read in the corrected track files, match up the tracks with their species and individual IDs, and reshape the data to work with the momentuHMM package.

In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
# load in csv giving the individual IDs for each track (exported from MySQL)
track_ids = pd.read_csv('../data/trackIDs.csv')

# load in csv giving the species IDs for each individual (exported from MySQL)
track_spps = pd.read_csv('../data/trackspps.csv')

In [6]:
# list of usable observations
# focal_obs = ['015', '027', '066', '074']
focal_obs = ['066', '074']

In [ ]:
for t in focal_obs:
    file = '../data/raw_tracks/tracks' + t + '_final.npy'
    tracks = np.load(file, allow_pickle = True)
    z,f,c = tracks.shape
    out_arr = np.column_stack((np.repeat(np.arange(z),f),tracks.reshape(z*f,-1)))
    out_df = pd.DataFrame(out_arr, columns = ['track_num', 'lon', 'lat'])
    out_df['frame'] = np.tile(np.arange(f),z)
    out_df['trackID'] = ['ob' + t + '-' + str(int(p)).zfill(4) for p in out_df.track_num]
    out_df['frame'] = np.tile(np.arange(f),z)
    out_df['trackID'] = ['ob' + t + '-' + str(int(p)).zfill(4) for p in out_df.track_num]
    out_df['indID'] = [track_ids[track_ids['trackID'] == p]['indID'].values[0] for p in out_df['trackID']]
    out_df['spp'] = [track_spps[track_spps['indID'] == p]['species'].values[0] for p in out_df['indID']]
    out_df['focal'] = [track_ids[track_ids['trackID'] == p]['focal'].values[0] for p in out_df['trackID']]
    clean_tracks = out_df[(out_df['spp'] == 'gz') | (out_df['spp'] == 'pz')]
    clean_tracks.dropna(inplace = True)
    name = 'ob' + t + '_tracks.csv'
    clean_tracks.to_csv(os.path.join('../data', 'tracks', name), index = False)

Each track file now has the coordinates for the animal's location in each frame matched up with the track ID, the individual animal ID, the species adn the focal status.

The tracks that are output here should be the "raw" data that are included with the paper dataset. That is, there is no reason for readers to have to run the code in this notebook themselves.